In [360]:
import pandas as pd
import pickle
from sklearn.utils import resample
import string
from nltk.stem import WordNetLemmatizer
import spacy
import re
import os
import nltk
import numpy as np
import itertools
from sklearn.utils import resample
from sklearn.metrics import classification_report
lemma = nltk.WordNetLemmatizer()
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
import math as mt
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
import re
import string
import numpy as np
from sklearn import metrics
from scipy.spatial.distance import cdist

In [361]:
data.shape

(2483, 6)

In [367]:
data=pd.read_csv('C:\\Users\\hp\\Downloads\\XH_chat_final.csv')
data.dropna(subset=['id'],inplace=True)

In [365]:
data.head()

,Unnamed: 0,id,Cluster_x,Subcluster,issue,full_chat
0,0,702106,Alarm_Issues,Others,my lights is out not cut off is my alarm still...,['my lights is out not cut off is my alarm sti...
1,1,702125,Keypad_Issues,Entry_delay_Alert,several times in the last couple of weeks I ha...,['several times in the last couple of weeks I ...
2,2,702381,Keypad_Issues,NaN,My alarm key pad is not working and I am scare...,['My alarm key pad is not working and I am sca...
3,3,702408,Password_Reset/Forget_Password,Manage_keypad_Codes,how do I set a temporary code to give to a dog...,['how do I set a temporary code to give to a d...
4,4,702971,Password_Reset/Forget_Password,Reset_Central_Station_Passcode,I have forgotten my central dispatch code. How...,['I have forgotten my central dispatch code. H...


# Model building for main labels

In [344]:
wordnet_lemmatizer = WordNetLemmatizer()
porter=PorterStemmer()
stop_words = stopwords.words('english')

def lemmatize(sentence):
    return ' '.join([lemma.lemmatize(word, 'v') for word in sentence.split(" ")])
                    
def clean(line):
    line = re.sub(r'[?,",:,.,/,!]', ' ', line)
    line = re.sub('[$]', 'Replaceddollar ', line)
    line= re.sub("[^a-zA-Z]", ' ', line)
    line=[i.lower() for i in line.split(' ') if i.lower() not in stop_words]  # Removing the stopwords excluding few neg words
    line=[wordnet_lemmatizer.lemmatize(i.lower(),pos="a") for i in line]
    line=[wordnet_lemmatizer.lemmatize(i.lower(),pos="n") for i in line]
    line=[wordnet_lemmatizer.lemmatize(i.lower(),pos="r") for i in line]
    line=[wordnet_lemmatizer.lemmatize(i.lower(),pos="v") for i in line]
    
    line=" ".join([porter.stem(j) for j in line if len(j)>1 and (j.isnumeric() is False)])
    line = re.sub('  +', ' ', line) # Removing the extra spaces
    return line.strip() #Returning the stripped sentence

In [48]:
data.columns

Index(['Unnamed: 0', 'id', 'Cluster_x', 'Subcluster', 'issue', 'full_chat'], dtype='object')

In [49]:
data['cleaned_text']=data['issue'].apply(lambda x: clean(x))

In [50]:
X_train,X_test,y_train,y_test=train_test_split(data['cleaned_text'], data['Cluster_x'])

In [51]:
data['Cluster_x'].value_counts()

Keypad_Issues                     884
Password_Reset/Forget_Password    855
Need_Security_Certificate         599
Alarm_Issues                      142
Name: Cluster_x, dtype: int64

In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer
# create the transform
vectorizer = TfidfVectorizer(sublinear_tf=True, min_df=3, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
#vectorizer = TfidfVectorizer(ngram_range=(1,2))
# tokenize and build vocab
vectorizer.fit(X_train)

TfidfVectorizer(encoding='latin-1', min_df=3, ngram_range=(1, 2),
                stop_words='english', sublinear_tf=True)

In [53]:
from sklearn.svm import LinearSVC
lsvm=LinearSVC()
lsvm.fit(vectorizer.transform(X_train),y_train)

LinearSVC()

In [54]:
y_pred=lsvm.predict(vectorizer.transform(X_test))

In [55]:
confusion_matrix(y_test,y_pred)

array([[ 21,  11,   0,   2],
       [  4, 221,   0,   5],
       [  0,   0, 139,   0],
       [  3,   7,   0, 207]], dtype=int64)

In [57]:
print(classification_report(y_test,y_pred))

                                precision    recall  f1-score   support

                  Alarm_Issues       0.75      0.62      0.68        34
                 Keypad_Issues       0.92      0.96      0.94       230
     Need_Security_Certificate       1.00      1.00      1.00       139
Password_Reset/Forget_Password       0.97      0.95      0.96       217

                      accuracy                           0.95       620
                     macro avg       0.91      0.88      0.90       620
                  weighted avg       0.95      0.95      0.95       620



In [32]:
data.head()

,Unnamed: 0,id,Cluster_x,Subcluster,issue,full_chat,cleaned_text
0,0,702106,Alarm_Issues,Others,my lights is out not cut off is my alarm still...,['my lights is out not cut off is my alarm sti...,light cut alarm still work
1,1,702125,Keypad_Issues,Entry_delay_Alert,several times in the last couple of weeks I ha...,['several times in the last couple of weeks I ...,sever time last coupl week disarm system main ...
2,2,702381,Keypad_Issues,NaN,My alarm key pad is not working and I am scare...,['My alarm key pad is not working and I am sca...,alarm key pad work scar night
3,3,702408,Password_Reset/Forget_Password,Manage_keypad_Codes,how do I set a temporary code to give to a dog...,['how do I set a temporary code to give to a d...,set temporari code give dog sitter weekend
4,4,702971,Password_Reset/Forget_Password,Reset_Central_Station_Passcode,I have forgotten my central dispatch code. How...,['I have forgotten my central dispatch code. H...,forget central dispatch code reset


# Model Bulding for Sub-labels

In [368]:
data['full_chat1']=data['full_chat'].apply(lambda x: ''.join(eval(x)))

In [370]:
data['full_chat1'][0]

"my lights is out not cut off is my alarm still workingXfinity Home Guide:Bot StartedIf you're having an emergency right now, please call 911. Dial 911 to stop a crime, report a fire, or save a life.\n\nWhat would you like more information on?Alarm in ProgressIf you're having an emergency right now, please call 911. Dial 911 to stop a crime, report a fire, or save a life.Thanks for contacting us!  It's been a while since we last heard from you.  We will be here to support you if you can chat with us again in the future.Bot Ended"

In [371]:
training_data=data[pd.notnull(data['Subcluster'])]
test_data=data[pd.isnull(data['Subcluster'])]

In [372]:
training_data.shape

(1329, 7)

In [373]:
test_data.shape

(1154, 7)

In [497]:
stop_sentences=["Xfinity Home Guide",
                "Bot Started",
                "If you're having an emergency right now",
                "please call 911",
                "Dial 911 to stop a crime",
                "report a fire",
                "or save a life",
                "Thanks for contacting us",
#                 "Confirm your Xfinity Home system is disarmed",
#                 "Working on devices while the system is armed can generate false alarms",
#                 "Remove the Keypad from the wall by sliding upwards until it comes free of the mounting bracket",
#                 "Removing Keypad from the mounting bracket will trigger a Tamper warning on the Touchscreen Controller",
#                 "This will go away once the Keypad is put back on the mounting bracket",
#                 "Remove and replace the batteries",
#                 "Replace the Keypad on the mounting bracket",
#                 "At this time, the Tamper warning disappears",
#                 "Arm the system",
                "Did that resolve your issue",
                "No, connect to a specialist",
                "We will be here to support you if you can chat with us again in the future",
                "Bot Ended",
                "It's been a while since we last heard from you",
                "You can initiate a new chat anytime during our chat hours (7am - 2am EDT)",
                "It seems like you are busy at this time"
               "Please acknowledge my last message","please call us at 1-800-XFINITY (1-800-934-6489)",
                "We will be here to support you if you can chat with us again in the future",
                "visit support.xfinity.com/chat for live agent support at any time",
                "If you need help outside of our chat business hours (7am -2am)",
                "It's been an hour since we last heard from you", "we will save & close this chat",
                "Thank you for choosing XFINITY",
                "It was my pleasure to assist you today",
                "Awesome","I apologize for the wait time in queue",
                "You are most welcome",
                "When you have a moment, please ensure your Emergency Contact information is up to date",
                "You can review and update these from the Xfinity Home mobile app and Subscriber Portal",
                "https://www.xfinity.com/support/home-security/add-change-emergency-contact-mobile-app",
                "https://www.xfinity.com/support/articles/manage-emergency-dispatch-contacts",
                "You have been great",
                "Before we connect you to a specialist",
                "Let me quickly help you in this concern",
                "All of our specialists are currently busy and will chat with you shortly",
                "Please make sure you have push notifications enabled to see your specialist's reply","I apologize for the inconvenience",
                "We value your business as a customer", "Have a great day ahead","Accepted the solution","Good Afternoon","Good Morning",
                "It was a pleasure to help","May I have your complete name","May I have your good name","May I have your full name",
                "Are we good so far","What would you like more information on","I hope you are doing good","Please be assured I will surely help you with this",
               "Keypad codes are used to access the Xfinity Home Security System while the Central Station passcode is a security feature that allows you to identify yourself or someone you trust to the Central Monitoring Station in the event of an alarm",
               "Connect to a specialist","sure","You're welcome","You are welcome","got it",'thanks',"Maintaining these is important in the event that an alarm occurs",
               "Thank you","thanks a lot","I appreciate your efforts and patience","I appreciate your cooperation and patience throughout the chat",
               "Have a great day","Hi","Hello","bye"]
stop_sentences=[i.lower() for i in stop_sentences]

In [409]:
# Taking all chat info

In [410]:
def remove_sentences(x):
    for i in stop_sentences:
        x=x.replace(i,"")
    return x    

In [411]:
training_data['full_chat2']=training_data['full_chat1'].apply(lambda x: remove_sentences(x))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [412]:
training_data['Subcluster']=training_data['Subcluster'].apply(lambda x: x.lower())

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [413]:
training_data['cleaned_full_text']=training_data['full_chat2'].apply(lambda x: clean(x))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [427]:
#new_bat=training_data[training_data['Subcluster']=="new_bat_not_working"].index.tolist()

In [456]:
#train_new_bat=new_bat[int(len(new_bat)/2):]

In [457]:
#train_new_bat

[2016, 2080, 2146, 2161, 2285, 2291, 2365]

In [458]:
#test_new_bat=new_bat[:int(len(new_bat)/2)]

In [459]:
#test_new_bat

[574, 1519, 1529, 1817, 1925, 1946, 2011]

In [480]:
#training_data1=training_data[training_data.index.isin(new_bat)==False]

In [481]:
training_data['Subcluster'].value_counts()

security_alarm_certificate        563
reset_master_code                 185
entry_delay_alert                  93
others                             88
reset_central_station_passcode     80
manage_keypad_codes                51
false_alarm                        38
arming/disarming                   34
keypad_pairing                     30
reset_keypad                       26
keypad_troubleshooting             24
reset_home_app_password            16
keypad_cont_blinking               15
comm_falure_alert                  14
entry_exit_delay_setting           12
panic_button_feature               11
new_keypad_enquiry                 11
low_bat_alert                       8
tamper_alert                        8
reset_secret_question               6
norton_subscription                 2
Name: Subcluster, dtype: int64

In [498]:
X_train,X_test,y_train,y_test=train_test_split(training_data['cleaned_full_text'], training_data['Subcluster'],stratify=training_data['Subcluster'],random_state=2,test_size=0.15)

In [486]:
#a=X_train.append(training_data[training_data.index.isin(train_new_bat)]['cleaned_full_text'])

In [487]:
#y=y_train.append(training_data[training_data.index.isin(train_new_bat)]['Subcluster'])

In [499]:
from sklearn.feature_extraction.text import TfidfVectorizer
# create the transform
vectorizer = TfidfVectorizer(sublinear_tf=True, min_df=3, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
#vectorizer = TfidfVectorizer(ngram_range=(1,2))
# tokenize and build vocab
vectorizer.fit(X_train)

TfidfVectorizer(encoding='latin-1', min_df=3, ngram_range=(1, 2),
                stop_words='english', sublinear_tf=True)

In [500]:
lsvc=LinearSVC()
lsvc.fit(vectorizer.transform(X_train),y_train)

LinearSVC()

In [492]:
#b=X_test.append(training_data[training_data.index.isin(test_new_bat)]['cleaned_full_text'])

In [493]:
#y1=y_test.append(training_data[training_data.index.isin(test_new_bat)]['Subcluster'])

In [501]:
y_pred=lsvc.predict(vectorizer.transform(X_test))

In [502]:
confusion_matrix(y_test,y_pred)

array([[ 3,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  1,  0,  0,  0,
         0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  1,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0],
       [ 0,  0, 14,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0],
       [ 0,  0,  0,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  4,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0,  0,
         0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  2,  0,  0,  0,
         0,  0,  0,  0,  0],
       [ 0,  0,  1,  0,  0,  0,  1,  2,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  6,  0,  0,  0,  0,  0,  0,
         0,  2,  0,

In [503]:
print(classification_report(y_test,y_pred))

                                precision    recall  f1-score   support

              arming/disarming       0.60      0.60      0.60         5
             comm_falure_alert       0.00      0.00      0.00         2
             entry_delay_alert       0.88      1.00      0.93        14
      entry_exit_delay_setting       1.00      0.50      0.67         2
                   false_alarm       0.80      0.67      0.73         6
          keypad_cont_blinking       1.00      1.00      1.00         2
                keypad_pairing       0.40      0.50      0.44         4
        keypad_troubleshooting       0.67      0.50      0.57         4
                 low_bat_alert       1.00      1.00      1.00         1
           manage_keypad_codes       1.00      0.75      0.86         8
           new_bat_not_working       0.00      0.00      0.00         2
            new_keypad_enquiry       1.00      1.00      1.00         2
                        others       0.59      0.77      0.67  

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
"Take out the batteries from affected Keypad",
"Click on Gear Icon on top right of touchscreen",
"Click on device option",
"Enter 4321 when asked for their Master Keypad Code",
"Please make sure you are entering 4321 only, not your own master code",
"Passphrase is",
"It will be valid for 24 hours"